In [46]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from wordcloud import WordCloud
import pickle
import re
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shrut\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


PreProcessing

In [19]:
df.shape

(1599999, 6)

In [20]:
col_names = ['target','id','date','flag','user','text']
df.columns = col_names

In [21]:
df.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [22]:
df.target.value_counts()

target
4    800000
0    799999
Name: count, dtype: int64

In [23]:
df.shape

(1599999, 6)

In [24]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv',names=col_names,encoding='ISO-8859-1')
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [25]:
df.shape

(1600000, 6)

In [26]:
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [27]:
df.target.value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [28]:
df.replace({'target':{4:1}},inplace=True)

In [29]:
df.target.value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

Remove the extra words and characters

In [30]:
port_stem = PorterStemmer()

In [31]:
def stemming(c):
    stemmed_content = re.sub('[^a-zA-Z]',' ',c).lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content        

In [34]:
df['stemmed_content'] = df['text'].apply(stemming)

In [35]:
df.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [36]:
df.target

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64

In [84]:
x = df['stemmed_content'].values
y = df['target'].values

Splitting the data

In [85]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [86]:
x_train.shape

(1280000,)

In [87]:
x_test.shape

(320000,)

Convert the text data into Numerical Form

In [88]:
vector = TfidfVectorizer()
x_train = vector.fit_transform(x_train)
x_test = vector.transform(x_test)

In [89]:
x_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453092 stored elements and shape (1280000, 461488)>

Import the model

In [90]:
model = LogisticRegression(max_iter= 1000)

In [91]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

Prediction on Training Data

In [92]:
prediction = model.predict(x_train)

In [93]:
accuracy = accuracy_score(y_true=y_train,y_pred=prediction)
print(f"Accuracy on Training data is : {accuracy}")

Accuracy on Training data is : 0.79871953125


Prediction on Test Data

In [94]:
x_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2289192 stored elements and shape (320000, 461488)>

In [96]:
test_prediction = model.predict(x_test)

In [97]:
accuracy = accuracy_score(y_true=y_test,y_pred=test_prediction)
print(f"Accuracy on Testing data is : {accuracy}")

Accuracy on Testing data is : 0.77668125


Save the Model

In [99]:
filename = "sentiment_analysis_model.sav"
pickle.dump(model,open(filename,'wb'))

In [100]:
load_model = pickle.load(open('sentiment_analysis_model.sav','rb'))

Prediction

In [101]:
y_test[100]

np.int64(0)

In [102]:
pre = load_model.predict(x_test[100])
pre

array([0])

In [103]:
if pre == 0:
    print("Negative Message")
else:
    print("Positive Message")

Negative Message
